4 Channel 

In [ ]:
// Create audio context
var audioCtx = new (window.AudioContext || window.webkitAudioContext)();

// Create Oscillators
var oscillators = [];
var gainNodes = [];
for (var i = 0; i < 4; i++) {
    var oscillator = audioCtx.createOscillator();
    var gainNode = audioCtx.createGain();

    oscillator.connect(gainNode);
    gainNode.connect(audioCtx.destination);

    oscillator.type = 'sine';
    oscillator.frequency.value = 440; // default frequency
    gainNode.gain.value = 0; // default amplitude

    oscillator.start();

    oscillators.push(oscillator);
    gainNodes.push(gainNode);
}

// Canvas setup
var canvas = document.getElementById('oscilloscope');
var ctx = canvas.getContext('2d');
var canvasWidth = canvas.width;
var canvasHeight = canvas.height;

// Animation
function animate() {
    requestAnimationFrame(animate);

    // Clear canvas
    ctx.clearRect(0, 0, canvasWidth, canvasHeight);

    // Draw each oscillator
    for (var i = 0; i < oscillators.length; i++) {
        var oscillator = oscillators[i];
        var gain = gainNodes[i].gain.value;

        ctx.beginPath();
        for (var x = 0; x < canvasWidth; x++) {
            var y = gain * 50 * Math.sin((2 * Math.PI) * (x / canvasWidth) * oscillator.frequency.value + audioCtx.currentTime) + canvasHeight / 2;
            ctx.lineTo(x, y);
        }
        ctx.stroke();
    }
}
animate();

// Function to update oscillator
function updateOscillator(index, frequency, amplitude) {
    oscillators[index].frequency.value = frequency;
    gainNodes[index].gain.value = amplitude;
}

+Analyser

In [ ]:
// Create audio context
var audioCtx = new (window.AudioContext || window.webkitAudioContext)();

// Create Oscillators
var oscillators = [];
var gainNodes = [];
var analyser = audioCtx.createAnalyser();
for (var i = 0; i < 4; i++) {
    var oscillator = audioCtx.createOscillator();
    var gainNode = audioCtx.createGain();

    oscillator.connect(gainNode);
    gainNode.connect(analyser);

    oscillator.type = 'sine';
    oscillator.frequency.value = 440; // default frequency
    gainNode.gain.value = 0; // default amplitude

    oscillator.start();

    oscillators.push(oscillator);
    gainNodes.push(gainNode);
}

analyser.connect(audioCtx.destination);

// Canvas setup
var canvas = document.getElementById('oscilloscope');
var ctx = canvas.getContext('2d');
var canvasWidth = canvas.width;
var canvasHeight = canvas.height;

// Animation
function animate() {
    requestAnimationFrame(animate);

    // Clear canvas
    ctx.clearRect(0, 0, canvasWidth, canvasHeight);

    // Get frequency data
    var data = new Uint8Array(analyser.frequencyBinCount);
    analyser.getByteFrequencyData(data);

    // Draw frequency data
    for (var i = 0; i < data.length; i++) {
        var value = data[i];
        var percent = value / 255;
        var height = canvasHeight * percent;
        var offset = canvasHeight - height - 1;
        var barWidth = canvasWidth / data.length;
        ctx.fillStyle = 'white';
        ctx.fillRect(i * barWidth, offset, barWidth, height);
    }
}
animate();

// Function to update oscillator
function updateOscillator(index, frequency, amplitude) {
    oscillators[index].frequency.value = frequency;
    gainNodes[index].gain.value = amplitude;
}

Single Channel Time Visual

In [ ]:
// Create audio context
var audioCtx = new (window.AudioContext || window.webkitAudioContext)();

// Create Oscillator and gainNode
var oscillator = audioCtx.createOscillator();
var gainNode = audioCtx.createGain();

oscillator.connect(gainNode);
gainNode.connect(audioCtx.destination);

oscillator.type = 'sine'; // sine wave — other values are 'square', 'sawtooth', 'triangle' and 'custom'
oscillator.frequency.value = 440; // value in hertz
oscillator.start();

// Create an analyser node
var analyser = audioCtx.createAnalyser();
gainNode.connect(analyser);

// Use fftSize property for the frequency domain
analyser.fftSize = 2048;
var bufferLength = analyser.frequencyBinCount;
var dataArray = new Uint8Array(bufferLength);

// Canvas context
var canvas = document.getElementById('canvas');
var canvasCtx = canvas.getContext('2d');

// Draw an oscilloscope of the current audio source
function draw() {
    requestAnimationFrame(draw);

    analyser.getByteTimeDomainData(dataArray);

    canvasCtx.fillStyle = 'rgb(200, 200, 200)';
    canvasCtx.fillRect(0, 0, canvas.width, canvas.height);

    canvasCtx.lineWidth = 2;
    canvasCtx.strokeStyle = 'rgb(0, 0, 0)';

    canvasCtx.beginPath();

    var sliceWidth = canvas.width * 1.0 / bufferLength;
    var x = 0;

    for(var i = 0; i < bufferLength; i++) {
        var v = dataArray[i] / 128.0;
        var y = v * canvas.height / 2;

        if(i === 0) {
            canvasCtx.moveTo(x, y);
        } else {
            canvasCtx.lineTo(x, y);
        }

        x += sliceWidth;
    }

    canvasCtx.lineTo(canvas.width, canvas.height / 2);
    canvasCtx.stroke();
}

draw();

Freq Analyser

In [ ]:
// Draw an oscilloscope of the current audio source
function draw() {
    requestAnimationFrame(draw);

    // Get frequency data
    var frequencyData = new Uint8Array(analyser.frequencyBinCount);
    analyser.getByteFrequencyData(frequencyData);

    // Clear canvas
    canvasCtx.fillStyle = 'rgb(200, 200, 200)';
    canvasCtx.fillRect(0, 0, canvas.width, canvas.height);

    // Draw frequency spectrum
    for (var i = 0; i < frequencyData.length; i++) {
        var value = frequencyData[i];
        var percent = value / 255;
        var height = canvas.height * percent;
        var offset = canvas.height - height - 1;
        var barWidth = canvas.width / frequencyData.length;

        canvasCtx.fillStyle = 'black';
        canvasCtx.fillRect(i * barWidth, offset, barWidth, height);
    }
}